In [2]:
!pip install spacy
!python -m spacy download es_core_news_sm
!pip install text2num


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\emont\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     --------------------------------------- 0.0/12.9 MB 162.5 kB/s eta 0:01:20
     --------------------------------------- 0.0/12.9 MB 186.2 kB/s eta 0:01:10
     --------------------------------------- 0.0/12.9 MB 217.9 kB/s eta 0:00:59
     --------------------------------------- 0.1/12.9 MB 297.7 kB/s eta 0:00:44
     --------------------------------------- 0.1/12.9 MB 392.2 kB/s eta 0:00:33
      -------------------------------------- 0.2/12.9 MB 581.0 kB/s eta 0:00:22
      -------------------------------------- 0.2/12.9 MB 719.7 kB/s eta 0:00:18
     - -------------------------------------- 0.4/12.9 MB 1.1 MB/s eta 0:00:12
     - -------------------------------------- 0.5/12.9 MB 1.3 MB/s eta 0:00:10
     -- ------------------------------------- 0.9/12.9 MB 2.0 MB/s eta 0:00:07
     --- ------------------------------------ 1.0/12.9 MB 2


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\emont\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/236.0 kB ? eta -:--:--
   --- ------------------------------------ 20.5/236.0 kB ? eta -:--:--
   ----- --------------------------------- 30.7/236.0 kB 445.2 kB/s eta 0:00:01
   ------ -------------------------------- 41.0/236.0 kB 393.8 kB/s eta 0:00:01
   ----------- --------------------------- 71.7/236.0 kB 438.9 kB/s eta 0:00:01
   ---------------- --------------------- 102.4/236.0 kB 538.9 kB/s eta 0:00:01
   -------------------------------- ----- 204.8/236.0 kB 831.5 kB/s eta 0:00:01
   -------------------------------------- 236.0/236.0 kB 851.9 kB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\emont\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
# -------------------------------------
# LIBRERÍAS Y CONFIGURACIÓN DE MODELOS
# -------------------------------------
import spacy
import json
from spacy.matcher import Matcher
from text_to_num import text2num

nlp = spacy.load("es_core_news_sm")
matcher = Matcher(nlp.vocab)


In [4]:
# ---------------------------------------
# CONFIGURACIÓN DE INFORMACIÓN UTILIZADA
# ---------------------------------------

MESES = [
    "enero", "febrero", "marzo", "abril", "mayo", "junio",
    "julio", "agosto", "septiembre", "octubre", "noviembre", "diciembre"
]

AEROLINEAS = [
    "iberia", "latam", "avianca", "copa", "american airlines", "delta", "qatar airways", "emirates", "ryanair", "jetblue",
    "aeromexico", "aeroregional", "air europa", "klm", "jetsmart", "united"
]

# --------------------------
# CONFIGURACIÓN DE MATCHERS
# --------------------------

# ---- FECHA ----
matcher.add(
    "FECHA",
    [[
        {"LIKE_NUM": True},
        {"LOWER": "de"},
        {"LOWER": {"IN": MESES}}
    ]]
)

# ---- AEROLÍNEAS ----
for aerolinea in AEROLINEAS:
    pattern = [{"LOWER": palabra} for palabra in aerolinea.split()]
    matcher.add("AEROLINEA", [pattern])


In [5]:
# ---------------------
# FUNCIONES AUXILIARES
# ---------------------
def normalizar_numero(texto):
  """
    Convierte cantidades expresadas en lenguaje natural o en formato numérico
    a un valor entero. Soporta números escritos en español (ej. "veinte y cinco")
    mediante la librería text2num, garantizando escalabilidad y consistencia
    en el dato final.
  """
  texto = texto.lower().strip()
  if texto.isdigit():
      return int(texto)

  try:
      return text2num(texto, "es")
  except:
      return None

def extraer_ciudad(doc, preposicion):
  """
    Extrae el nombre completo de una ciudad, siendo compatible con ciudades
    formadas por varias palabras y con preposiciones internas, como "La Paz",
    "Buenos Aires", "Ciudad de México" o "Río de Janeiro".
  """
  ARTICULOS = {"la", "el", "los", "las"}
  for i, token in enumerate(doc):
      if token.lower_ == preposicion:
          ciudad_tokens = []
          j = i + 1

          if j < len(doc) and doc[j].lower_ in ARTICULOS:
              ciudad_tokens.append(doc[j].text)
              j += 1

          if j < len(doc) and doc[j].pos_ == "PROPN":
              ciudad_tokens.append(doc[j].text)
              j += 1
          else:
              continue

          while (
              j + 1 < len(doc)
              and doc[j].lower_ == "de"
              and doc[j + 1].pos_ == "PROPN"
          ):
              ciudad_tokens.append(doc[j].text)
              ciudad_tokens.append(doc[j + 1].text)
              j += 2

          return " ".join(ciudad_tokens)

  return None


def extraer_cantidad(doc):
  """
    Identifica y extrae la cantidad de boletos a partir del contexto del mensaje,
    analizando las palabras que preceden a términos como "boletos" o "pasajes".
    Permite interpretar números simples y compuestos en español
    (ej. "cuatro", "veinte y cinco") y los normaliza a valores enteros.
  """
  PALABRAS_CANTIDAD = {"boletos", "boleto", "billetes", "billete", "pasajes", "pasaje"}
  for i, token in enumerate(doc):
      if token.lower_ in PALABRAS_CANTIDAD:
          numero_tokens = []
          j = i - 1

          while j >= 0 and (
              doc[j].like_num or doc[j].lower_ == "y"
          ):
              numero_tokens.insert(0, doc[j].text)
              j -= 1

          numero_texto = " ".join(numero_tokens).strip()

          if numero_texto.isdigit():
              return int(numero_texto)

          try:
              return text2num(numero_texto, lang="es")
          except:
              return None

  return None

def extract_information(text):
  """
  Función principal de procesamiento de lenguaje natural.
  Integra el análisis del texto del usuario para extraer información estructurada
  como origen, destino, fecha, cantidad y aerolínea.
  """
  doc = nlp(text)

  origen = extraer_ciudad(doc, "de")
  destino = extraer_ciudad(doc, "a")
  cantidad = extraer_cantidad(doc)
  if cantidad is None:
    cantidad = 1

  fecha = aerolinea = None

  for match_id, start, end in matcher(doc):
      label = nlp.vocab.strings[match_id]
      span = doc[start:end]

      if label == "FECHA" and fecha is None:
          fecha = span.text

      elif label == "AEROLINEA" and aerolinea is None:
          aerolinea = span.text.title()

  return {
      "origen": origen,
      "destino": destino,
      "fecha": fecha,
      "cantidad": cantidad,
      "aerolínea": aerolinea
  }


In [8]:
# ------------------
# ASISTENTE VIRTUAL
# ------------------
def assistant():
    print("Hola, bienvenido a BMJ.SA\n¿Cómo te puedo ayudar?")
    user_input = input(">> ")
    print(">> ",user_input)
    info = extract_information(user_input)

    if info["fecha"] is None:
      print(
          f"\nPerfecto, comienzo la búsqueda de tu viaje a {info['destino']} "
          f"desde {info['origen']} con {info['aerolínea']}."
      )
    else:
      print(
          f"\nPerfecto, comienzo la búsqueda de tu viaje a {info['destino']} "
          f"desde {info['origen']} para el {info['fecha']} "
          f"con {info['aerolínea']}."
      )

    print("\nJSON generado:")
    print(json.dumps(info, indent=4, ensure_ascii=False))


assistant()

Hola, bienvenido a BMJ.SA
¿Cómo te puedo ayudar?
>>  Hola, quiero siete boletos para el 19 de marzo de Bilbao a Coruña con Iberia

Perfecto, comienzo la búsqueda de tu viaje a Coruña desde Bilbao para el 19 de marzo con Iberia.

JSON generado:
{
    "origen": "Bilbao",
    "destino": "Coruña",
    "fecha": "19 de marzo",
    "cantidad": 7,
    "aerolínea": "Iberia"
}
